In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

2022-12-08 15:31:55.685788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 15:31:55.793219: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 15:31:55.793231: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 15:31:56.258633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
df_o = pd.read_csv("nfl_pass_rush_receive_raw_data.csv")
df_o = df_o.drop(['game_id', 'Team_abbrev', 'pass_yds_bonus', 'rush_yds_bonus', 'rec_yds_bonus', 'Total_DKP', 'Off_DKP', 'Total_FDP', 'Off_FDP', 'Total_SDP', 'Off_SDP', 'offense', 'off_pct', 'vis_score', 'home_score', 'OT', 'Roof', 'Surface', 'Temperature', 'Humidity', 'Wind_Speed', 'Vegas_Line', 'Vegas_Favorite', 'Over_Under'], 1)

/tmp/ipykernel_193195/520090854.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_o = df_o.drop(['game_id', 'Team_abbrev', 'pass_yds_bonus', 'rush_yds_bonus', 'rec_yds_bonus', 'Total_DKP', 'Off_DKP', 'Total_FDP', 'Off_FDP', 'Total_SDP', 'Off_SDP', 'offense', 'off_pct', 'vis_score', 'home_score', 'OT', 'Roof', 'Surface', 'Temperature', 'Humidity', 'Wind_Speed', 'Vegas_Line', 'Vegas_Favorite', 'Over_Under'], 1)


In [3]:
def str_to_date(s):
    return datetime.datetime(*list(map(int, s.split('-'))))
str_to_date = np.vectorize(str_to_date)

In [4]:
def get_days(t):
    return t.days
get_days = np.vectorize(get_days)

In [5]:
start = str_to_date(df_o.iloc[0]['game_date'])
weeks_o = get_days(str_to_date(df_o['game_date']) - start) // 7 + 1
df_o['week'] = weeks_o
df_o = df_o.drop('game_date', 1)

/tmp/ipykernel_193195/1957130345.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_o = df_o.drop('game_date', 1)


In [6]:
ids = df_o['player_id'].unique()
num_players_o = len(ids)
id_to_num = dict(zip(ids, np.arange(num_players_o)))
num_weeks = weeks_o[-1]
has_week_o = np.zeros([num_players_o, num_weeks])
for id, week in zip(df_o['player_id'], df_o['week']):
    has_week_o[id_to_num[id], week - 1] = 1

In [7]:
id_to_info = {}
for id, name, pos, team in zip(df_o['player_id'], df_o['player'], df_o['pos'], df_o['team']):
    id_to_info[id] = (name, pos, team)

In [8]:
df_new = [df_o]
for id in ids:
    for i in range(num_weeks):
        if has_week_o[id_to_num[id], i] == 0:
            df_new.append(pd.DataFrame([[id, '', id_to_info[id][0]] + [''] + [0] * 24 + ['']+ [0] * 13 + [''] * 2 + [i + 1]], columns=df_o.columns))
df_o = pd.concat(df_new)
df_o.to_csv("offense_cleaned.csv", index=False)

In [9]:
df_o.columns[27]

'comb_rush_play'

In [10]:
df_new = []
for id, (name, pos, team) in id_to_info.items():
    df_new.append(pd.DataFrame([[id, name, pos, team if len(team) == 3 else '']], columns=['player_id', 'player', 'pos', 'team']))
df_player_o = pd.concat(df_new)
df_player_o.to_csv("offense_players_cleaned.csv", index=False)

In [11]:
df_k = pd.read_csv('nfl_kicking_raw_data.csv')
df_k = df_k.drop(['game_id', 'Total_DKP', 'Total_FDP', 'Total_SDP', 'vis_score', 'home_score', 'OT', 'Roof', 'Surface',
       'Temperature', 'Humidity', 'Wind_Speed', 'Vegas_Line', 'Vegas_Favorite',
       'Over_Under'], 1)

/tmp/ipykernel_193195/1955825311.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_k = df_k.drop(['game_id', 'Total_DKP', 'Total_FDP', 'Total_SDP', 'vis_score', 'home_score', 'OT', 'Roof', 'Surface',


In [12]:
weeks_k = get_days(str_to_date(df_k['game_date']) - start) // 7 + 1
df_k['week'] = weeks_k
df_k = df_k.drop('game_date', 1)

/tmp/ipykernel_193195/3774448750.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_k = df_k.drop('game_date', 1)


In [13]:
names = df_k['player'].unique()
num_players_k = len(names)
name_to_num = dict(zip(names, np.arange(num_players_k)))
has_week_k = np.zeros([num_players_k, num_weeks])
for name, week in zip(df_k['player'], df_k['week']):
    has_week_k[name_to_num[name], week - 1] = 1

In [14]:
name_to_team = {}
for name, team in zip(df_k['player'], df_k['Off_abbrev']):
    name_to_team[name] = team

In [15]:
df_new = [df_k]
for name in names:
    for i in range(num_weeks):
        if has_week_k[name_to_num[name], i] == 0:
            df_new.append(pd.DataFrame([[name_to_team[name], ''] + [0] * 10 + [name] + [''] * 2 + [i + 1]], columns=df_k.columns))
df_k = pd.concat(df_new)
df_k.to_csv('kicking_cleaned.csv', index=False)

In [16]:
df_player_k = pd.DataFrame.from_dict(name_to_team, 'index').reset_index()
df_player_k.columns = ['player', 'Off_abbrev']
df_player_k.to_csv('kicking_players_cleaned.csv', index=False)